In [1]:
import os

In [2]:
%pwd

'c:\\Projects\\Kidney-disease-classification\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path as path
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: path
    training_model_path: path
    updated_base_model_path: path
    training_data : path
    params_eposhs :int
    params_batch_size :int
    params_is_augmentation :bool
    params_images_size :int


In [5]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories 
import tensorflow as tf

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=config_file_path,
                 params_filepath=params_file_path):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    def get_training_config(self) -> TrainingConfig:
        training =  self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data= os.path.join(self.config.data_ingestion.unzip_dir,"kidney-ct-scan-image")
        create_directories(
            [Path(training.root_dir)]
        )
        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            training_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_eposhs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_images_size=params.IMAGE_SIZE
        )
        return training_config
    

In [7]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [22]:
class Training:
    def __init__(self,config:TrainingConfig):
        self.config = config
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.2
       
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_images_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
            class_mode="sparse"
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.train_valid_generator= valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=20,
                horizontal_flip=True,
                vertical_flip=True,
                width_shift_range=0.1,
                height_shift_range=0.1,
                shear_range=0.1,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)    
    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.train_valid_generator.samples // self.train_valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_eposhs,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.train_valid_generator,
            validation_steps=self.validation_steps,
            
        )

        self.save_model(path=self.config.training_model_path, model=self.model)

        




        
        

In [23]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    
    raise e

Found 1471 images belonging to 2 classes.
Found 5889 images belonging to 2 classes.
368/368 [==============================] - 479s 1s/step - loss: 9.9607 - accuracy: 0.6889 - val_loss: 3.6341 - val_accuracy: 0.9437
